# UR5 Simulation forward kinematics
**Author** Eduardo Gomez

Using the Denavit-Hartenberg parameters of the UR5, program the forward kinematics

In [5]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd
import utils

## Setup Data

Here we will set up the data for the Denavit-Hartenberg parameters. The data was from: ...

In [ ]:
# First one I think is wrong
utils.



The matriz transformations can be expressed like these: 

$$
H_i^{i-1} = Rot_z(\theta_i)T(0,0,d_i)T(a_i, 0, 0)Rotx(\alpha_i)
$$

Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,

Here we'll define some utility functions

# Transformation matrix calcs

Because for the moment, all $\theta = 0$, the final coord is in the final matrix

## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [3]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')

print(sim)

<class 'zmqRemoteApi.sim'>


Here, we set up the id for controlling the robot

In [11]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [12]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

In [13]:
def reset_arm():
    """
    Resets the arm position to 0 deg
    """
    for j in joint_id:
        sim.setJointTargetPosition(j, np.deg2rad(0))

In [22]:
sim.startSimulation()
reset_arm()

In [15]:
reset_arm()

In [16]:

print(sim.getObjectPosition(connection_id, sim.handle_world))
angles = [0 for _ in range(6)]
# angles[1] = 90
angles = [np.deg2rad(a) for a in angles]
assert len(angles) == no_joints

for a, j in zip(angles, joint_id):
    sim.setJointTargetPosition(j, a)

sleep(1)

for idx, j in enumerate(joint_id):
    coord = sim.getObjectPosition(j, sim.handle_world)
    print(f"j{idx}: {np.round(coord,4)}")

coord = sim.getObjectPosition(connection_id, sim.handle_world)
print(f"cn: {np.round(coord,4)}")



[-0.1850380369714104, -5.254494075357469e-06, 0.9780366418975974]
j0: [ 0. -0.  0.]
j1: [-0.0703 -0.      0.066 ]
j2: [-0.0703 -0.      0.4911]
j3: [-0.0704 -0.      0.8833]
j4: [-0.1101 -0.      0.9289]
j5: [-0.0957 -0.      0.978 ]
cn: [-0.185 -0.     0.978]


Here we stop the simulation

In [29]:

print("Final coordinate in simulation")
print_rounded(coord)
print("Final coordinate according to method")
print_rounded(H_total[0:3,3])


Final coordinate in simulation
[-0.185 -0.     0.978]
Final coordinate according to method
[-0.0397  0.8665  0.0661]


In [21]:
sim.stopSimulation()

1